In [1]:
#tbl = '/mnt/e/rms/equipasaude/areainf_cs_2021_v3.xlsx'
tbl = '/mnt/e/rms/equipasaude/AInf_CSP_Azores_v3.xlsx'

tbl_sheet = 'ainf'

cscol = 'idcs'

fregcol = 'ddccff'

facilities     = '/mnt/e/rms/equipasaude/cs_azores.shp'
idfac = 'csid'

incidents      = '/mnt/e/rms/azores/rms_bgri21_pnt.shp'
fregpk = 'idfreg'

outfolder = '/mnt/e/rms/acc_csazores'

In [2]:
import os
import numpy as np
from glass.rd      import tbl_to_obj
from glass.rd.shp  import shp_to_obj
from glass.pys.oss import mkdir
from glass.wt.shp  import df_to_shp

/home/jasp/.virtualenvs/genv/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
df = tbl_to_obj(tbl, sheet=tbl_sheet)

In [4]:
df[fregcol] = df[fregcol].astype(str)
df[f"_{fregcol}"] = df[fregcol].str.replace('\t', '')

df[fregcol] = np.where(
    df[f"_{fregcol}"].isna(),
    df[fregcol], df[f"_{fregcol}"]
)

In [5]:
fregs = df[fregcol].unique()

In [6]:
fregs_cs = {}
fregs_ia = {}

for freg in fregs:
    fcs = df[df[fregcol] == freg]
    
    fregs_cs[freg] = list(fcs[cscol].unique())
    
    fregs_cs[freg].sort()
    
    fregs_ia[freg] = 0

In [7]:
# Get Influence Areas

iareas = {}

iaid = 1
for i in range(len(fregs)):
    if fregs_ia[fregs[i]]:
        continue
    
    iareas[iaid] = [fregs[i]]
    
    fregs_ia[fregs[i]] = 1
    
    for e in range(i+1, len(fregs)):
        if fregs_ia[fregs[e]]:
            continue
        
        if fregs_cs[fregs[i]] == fregs_cs[fregs[e]]:
            iareas[iaid].append(fregs[e])
            
            fregs_ia[fregs[e]] = 1
            
            continue
    
    iaid += 1

In [8]:
# For each iarea, get incidents and facilities

# Obter as facilities
fdf = shp_to_obj(facilities)

# Get incidents
idf = shp_to_obj(incidents)

In [9]:
for ia in iareas:
    # Select incidents
    s_idf = idf[idf[fregpk].isin(iareas[ia])]
    
    # Select facilities
    s_fdf = fdf[fdf[idfac].isin(fregs_cs[iareas[ia][0]])]
    
    if not s_idf.shape[0] or not s_fdf.shape[0]:
        print('Start Warning')
        print(ia)
        print(iareas[ia])
        print(fregs_cs[iareas[ia][0]])
        print('===============\n===========\n')
    
    # Make dir
    iafolder = mkdir(os.path.join(outfolder, f'ia_{str(ia)}'))
    
    # Save shapefiles
    df_to_shp(s_idf, os.path.join(iafolder, 'incidents.shp'))
    df_to_shp(s_fdf, os.path.join(iafolder, 'facilities.shp'))